In [1]:
# Import necessary libraries
import polars as pl
import psycopg2
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Establish database connection using TCP/IP
conn = psycopg2.connect(
    dbname=os.getenv("gsa_advantage"),
    user=os.getenv("cody"),
    password=os.getenv("686400Cj$"),
    host=os.getenv("170.187.201.28"),  # Default to localhost if DB_HOST is not set
    port=os.getenv("DB_PORT", "5432")        # Default to 5432 if DB_PORT is not set
)

# Define the query
query = """
    WITH reference_items AS (
        -- Get 100 random items from the specific contract
        SELECT *
        FROM gsa_product_extract_jan2024
        WHERE contract_number = '47QSEA20D003B'
        ORDER BY RANDOM()  -- Randomize the selection of the 100 items
        LIMIT 100
    ),
    competitor_items AS (
        -- Get all items with the same manufacturer_part_number from different contracts
        SELECT *
        FROM gsa_product_extract_jan2024 gi
        WHERE gi.contract_number != '47QSEA20D003B'  -- Ensure items are from different contracts
        AND gi.manufacturer_part_number IN (SELECT manufacturer_part_number FROM reference_items)
    )
    -- Combine reference items with their matches
    SELECT ref.*, 'reference' AS source
    FROM reference_items ref
    UNION ALL
    SELECT comp.*, 'competitor' AS source
    FROM competitor_items comp
    ORDER BY jprod_id;
    """

# Execute the query and load the data into a DataFrame
df = pl.read_database(query, conn)

# Close the database connection
conn.close()

# Display the first few rows of the DataFrame
df.head()

OperationalError: could not connect to server: No such file or directory
	Is the server running locally and accepting
	connections on Unix domain socket "/var/run/postgresql/.s.PGSQL.5432"?
